PROYECTO INDIVIDUAL UNO | SISTEMA DE RECOMENDACIÓN | /USUARIO614

In [1]:
import pandas as pd
import ast
import numpy as np
import json
import re

def load_json_lines(file_path):
    data = []
    with open(file_path, "r", encoding = 'utf-8') as file:
     for line in file:
        data.append(ast.literal_eval(line))
    return pd.DataFrame(data)


In [2]:
#Cargamos los archivos

df_games = pd.read_json(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\output_steam_games.json", lines=True)
df_reviews = load_json_lines(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\australian_user_reviews.json")
df_items = load_json_lines(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\australian_users_items.json")

ETL REVIEWS

In [3]:
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
#Importamos la librería a usar para realizar el análisis de sentimientos

from textblob import TextBlob

In [5]:
#Creamos la función para realizar el análisis de sentimientos
def sentiment_analysis(review):
    if isinstance(review, list) and len(review) > 0:
        text = review[0].get('review', '')
        sentiment = TextBlob(text).sentiment.polarity
        
        if sentiment < -0.2:
            return 0 # Negativo
        elif sentiment >= -0.2 and sentiment <= -0.2:
            return 1 # Neutral
        else:
            return 2 # Positivo
    else:
        return 1 # Valor que se asume cuando la reseña está ausente

In [6]:
#Aplicamos la función creada a la coliumna reviwes, para realizar el análisis y generaer la columna solicitada Sentiment_analysis
df_reviews['sentiment_analysis'] = df_reviews['reviews'].apply(sentiment_analysis)

In [7]:
#Desanidamos REVIEWS

#Creamos una lista vacía donde vamos a almacenar los datos de la columna 'reviews'
data_reviews = []

In [8]:
for index, fila in df_reviews.iterrows():
    for review in fila['reviews']:
        # Verificar si 'review' es un diccionario
        if isinstance(review, dict):
            data_reviews.append({
                "user_id": fila["user_id"],
                "user_url": fila["user_url"],
                "sentiment_analysis": fila["sentiment_analysis"],
                "funny": review.get("funny", ""),
                "posted": review.get("posted", ""),
                "last_edited": review.get("last_edited", ""),  
                "item_id": review.get("item_id", ""),
                "helpful": review.get("helpful", ""),
                "recommend": review.get("recommend", ""),
                "review": review.get("review", "")
            })

# Convertimos la lista de datos en un nuevo DataFrame
df_reviews_new= pd.DataFrame(data_reviews)

# Mostramos el DataFrame resultante
df_reviews_new.head(5)

,user_id,user_url,sentiment_analysis,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,2,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,2,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [9]:
#Eliminamos las columnas innecesarias
df_reviews_new.drop(['helpful','funny','last_edited','review'], axis=1, inplace=True)

In [10]:
#Eliminamos los puntos de la columna 'posted'
df_reviews_new['posted'] = df_reviews_new['posted'].str.replace('.', '')
df_reviews_new['posted'] = df_reviews_new['posted'].str.replace(',', '')
df_reviews_new['posted'] = df_reviews_new['posted'].str.replace('Posted', '')

In [11]:
# Función para comprobar y transformar el formato de las fechas
def analizar_fechas(df, columna):
    for index, fecha in enumerate(df[columna]):
        # Usar una expresión regular para verificar el formato deseado (dd-mm-AAAA)
        if not re.match(r'\d{2}-\d{2}-\d{4}', fecha):
            # Si no cumple con el formato, transformarlo en el formato correcto
            try:
                fecha_obj = pd.to_datetime(fecha, errors='raise')
                fecha_transformada = fecha_obj.strftime('%d-%m-%Y')
                df.at[index, columna] = fecha_transformada
            except ValueError:
                # Si no se puede transformar en una fecha válida, dejarlo como está
                pass

# Llamamos a la función para verificar y transformar las fechas
analizar_fechas(df_reviews_new, 'posted')

# Imprimimos el DataFrame resultante
print(df_reviews_new['posted'])

0        05-11-2011
1        15-07-2011
2        21-04-2011
3        24-06-2014
4        08-09-2013
            ...    
59300       July 10
59301        July 8
59302        July 3
59303       July 20
59304        July 2
Name: posted, Length: 59305, dtype: object


In [12]:
df_reviews_new['posted'] = pd.to_datetime(df_reviews_new['posted'], format='mixed', dayfirst=True, errors='coerce')


In [13]:
# Convertimos la columna 'posted' a tipo string (en caso que no lo sea)
df_reviews_new['posted'] = df_reviews_new['posted'].astype(str)

# Verificamos si todos los valores en 'posted' tienen el mismo formato (dd-mm-AAAA)
formato_esperado = r'\d{2}-\d{2}-\d{4}'  # Expresión regular para el formato dd-mm-AAAA

df_reviews_new.head(5)

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-11-05,1250,True
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-07-15,22200,True
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-04-21,43110,True
3,js41637,http://steamcommunity.com/id/js41637,2,2014-06-24,251610,True
4,js41637,http://steamcommunity.com/id/js41637,2,2013-09-08,227300,True


In [14]:
#Eliminamos los registros de fecha con dato NaN
df_reviews_new=df_reviews_new.dropna(subset=['posted'])
df_reviews_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   user_url            59305 non-null  object
 2   sentiment_analysis  59305 non-null  int64 
 3   posted              59305 non-null  object
 4   item_id             59305 non-null  object
 5   recommend           59305 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 2.3+ MB


In [15]:
#Transformamos la columna 'recommend'
df_reviews_new['recommend'] = df_reviews_new['recommend'].replace({True:1, False:0})

In [16]:
#Eliminamos duplicados
df_reviews_new=df_reviews_new.drop_duplicates(subset=['user_id'])

In [17]:
df_reviews_new.head(5)

,user_id,user_url,sentiment_analysis,posted,item_id,recommend
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,2011-11-05,1250,1
3,js41637,http://steamcommunity.com/id/js41637,2,2014-06-24,251610,1
6,evcentric,http://steamcommunity.com/id/evcentric,2,NaT,248820,1
12,doctr,http://steamcommunity.com/id/doctr,2,2013-10-14,250320,1
18,maplemage,http://steamcommunity.com/id/maplemage,2,2014-04-15,211420,1


In [18]:
#Guardamos el archivo
df_reviews_new= df_reviews_new.to_csv(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\df_reviews_final.csv", index=False)

ETL STEAM_GAMES

In [19]:

#Leemos y convertimos el archivo json a dataframe
with open(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\output_steam_games.json", encoding='utf-8') as f:
    data = f.readlines()

In [20]:
data = map(lambda x: x.rstrip(), data)
data_json_str = "[" + ','.join(data) + "]"
df_games = pd.read_json(data_json_str)

C:\Users\jotad\AppData\Local\Temp\ipykernel_20996\4193039845.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_games = pd.read_json(data_json_str)


In [21]:
df_games.head(2)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [22]:
#Eliminamos las columnas que no se necesitan para el procesamiento
#df_games = df_games.drop(columns = ['user_id','discount_price','items','steam_id','items_count'])

In [23]:
#Eliminamos los registros con valor nulo en todos los campos
df_games.dropna(how='all', inplace=True)

In [24]:
#Convertimos en 0 los registros que no tienen id
df_games[id]=df_games['id'].fillna(0).astype(int)
df_games[id]=df_games['id'].fillna(0)

In [25]:
#Generación de la variable año desde release_date
#Se da formato fecha a los valores de la columna release_date, quedando como nulos los que no tienen el formato
df_games['release_date']=pd.to_datetime(df_games['release_date'], errors='coerce')

#Se asegura que los datos cumplan con el formato AAAA-MM-DD
df_games['release_date']=pd.to_datetime(df_games['release_date']).dt.strftime('%Y-%m-%d')

#Se identifican los valores nulos de la columna release_date y se eliminan
df_games['release_date'].isnull().sum()
df_games = df_games.dropna(subset=['release_date'])

#Se crea la variable year
df_games['release_date']=pd.to_datetime(df_games['release_date'])
df_games['year']=df_games['release_date'].dt.year

#Se verifican los cambios
df_games[['release_date','year']].tail(15)

,release_date,year
120428,2018-01-03,2018
120429,2018-01-03,2018
120430,2016-11-19,2016
120431,2018-01-03,2018
120432,2018-01-03,2018
120434,2018-01-03,2018
120435,2018-10-01,2018
120436,2018-01-04,2018
120437,2018-01-04,2018
120438,2018-01-04,2018


In [26]:
#Transformamos la columna price
#Reemplazamos los valores free a 0
df_games['price'] = df_games['price'].replace(['Free to Play','Free Demo','Free HITMANâ„¢ Holiday Pack','Play the Demo','Free Mod','Play WARMACHINE: Tactics Demo','Free To Play','Free'], 0)

In [27]:
#Reemplazamos los registros de price que noi tiene relación con el precio por NaN
df_games['price'] = df_games['price'].replace(['Install Theme','Third-party','Starting at $','Starting at $','Play Now','Starting at $499.00','Starting at $449.00'], 'NaN')

In [28]:
#Eliminamos los regsitros NaN de price
df_games.dropna(subset=['price'])

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,2774905896864,year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,761140.0,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0.0,643980.0,Secret Level SRL,643980.0,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0,0.0,670290.0,Poolians.com,670290.0,2017
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域,767400.0,2017
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,0.0,772540.0,Trickjump Games Ltd,772540.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,Kebab it Up!,http://store.steampowered.com/app/745400/Kebab...,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",http://steamcommunity.com/app/745400/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",1.99,0.0,745400.0,Bidoniera Games,745400.0,2018
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS""",773640.0,2018
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada,733530.0,2018
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,610660.0,2018


In [29]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29783 entries, 88310 to 120443
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   publisher      23793 non-null  object        
 1   genres         28549 non-null  object        
 2   app_name       29782 non-null  object        
 3   title          29782 non-null  object        
 4   url            29783 non-null  object        
 5   release_date   29783 non-null  datetime64[ns]
 6   tags           29622 non-null  object        
 7   reviews_url    29782 non-null  object        
 8   specs          29114 non-null  object        
 9   price          28782 non-null  object        
 10  early_access   29783 non-null  float64       
 11  id             29782 non-null  float64       
 12  developer      28533 non-null  object        
 13  2774905896864  29783 non-null  float64       
 14  year           29783 non-null  int32         
dtypes: datetime64[ns](1

In [30]:
#Guardamos el archivo final
df_steam_final = df_games.to_csv(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\steam_games_final.csv", index=False)

ETL USER_ITEMS

In [31]:
#Cargamos el archivo json y lo convertimos en dataframe
def l_a_c():
    with open(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\australian_users_items.json",'rt', encoding='utf-8') as file:
        return[ast.literal_eval(line.strip())
            for line in file]
        
datos = l_a_c()

df_items = pd.DataFrame(datos)

df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [32]:
#Fragmentamos el archivo para poder desanidarlo sin problemas con el kernel

chunk_size = 1000  # Elige el tamaño del fragmento

# Dividimos el DataFrame en fragmentos
fragmentos = [df_items[i:i + chunk_size] for i in range(0, len(df_items), chunk_size)]

In [33]:
#Creamos la funcion para desanidar

import pandas as pd

def de_nest_items(df):
    # Explotamos la columna 'items' en múltiples filas
    items = df['items'].explode()

    # Eliminamos la columna 'items' original
    df = df.drop('items', axis=1)

    # Concatenamos los datos de 'items' con el DataFrame original
    df = pd.concat([df, items.apply(pd.Series)], axis=1)

    return df

In [34]:
fragmentos_desanidados = [de_nest_items(fragmento) for fragmento in fragmentos]

In [35]:
df_final = pd.concat(fragmentos_desanidados)

In [36]:
df_final

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,0
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0,NaN
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0,NaN
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0,NaN


In [37]:
#Eliminamos las columnas que no nos son necesarias
df_final.drop([0,'steam_id','playtime_2weeks'], axis=1, inplace=True)

In [38]:
#Eliminamos los registros con valor nulo en todos los campos
df_final.dropna(how='all', inplace=True)

In [39]:
#Eliminamos duplicados
df_final=df_final.drop_duplicates()

In [40]:
#Generamos una muestra del dataframe para probar las funciones
df_final_sample = df_final.sample(50000)

In [41]:
df_final_sample

,user_id,items_count,user_url,item_id,item_name,playtime_forever
18773,jadziatuvok,823,http://steamcommunity.com/id/jadziatuvok,283410,CT Special Forces: Fire for Effect,0.0
3963,rhysdog,101,http://steamcommunity.com/id/rhysdog,208090,Loadout,207.0
5665,Hirutha,171,http://steamcommunity.com/id/Hirutha,383080,Sakura Clicker,2139.0
26758,fereleth,874,http://steamcommunity.com/id/fereleth,1230,Mare Nostrum,0.0
23004,76561198006541747,126,http://steamcommunity.com/profiles/76561198006...,202090,Magicka: Wizard Wars,0.0
...,...,...,...,...,...,...
46387,76561198015238503,31,http://steamcommunity.com/profiles/76561198015...,12900,Audiosurf,41.0
49426,Chrazzari,136,http://steamcommunity.com/id/Chrazzari,380,Half-Life 2: Episode One,2.0
64119,76561198068611843,29,http://steamcommunity.com/profiles/76561198068...,274940,Depth,106.0
34515,76561198046755491,184,http://steamcommunity.com/profiles/76561198046...,10090,Call of Duty: World at War,409.0


In [42]:
#Guardamos la muestra en un archivo parquet
df_final_sample = df_final_sample.to_parquet(r"C:\Users\jotad\OneDrive\Escritorio\proyecto\df_final_sample.parquet", index=False)

c:\Users\jotad\AppData\Local\Programs\Python\Python310\lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
